In [1]:
class Auteur:
    def __init__(self, nom = ""):
        self.nom = nom
        self.uri = ""
        self.bio = ""
        self.genres = []
        self.site = ""
        self.pseudonyme = ""

class Oeuvre:
    def __init__(self, id, titre = " "):
        self.id = id
        self.titre = titre
        self.auteurs = []
        self.lieuPublication = ""
        self.editeur = ""
        self.annee = ""
        self.isbn = ""
        

In [2]:
auteurs = []
oeuvres = []

In [7]:

f = open('../Data/ILE/auteurs_ILE.csv',encoding="UTF-8")
document = f.read().split('\n-----\n')
for item in document:
    auteur = Auteur()
    if item:
        infos = item.split('<>')
        auteur.uri = infos[0]
        auteur.nom = infos[1]
        auteur.genres = infos[2].split(',')
        if infos[3] != ' ':
            auteur.pseudonyme = infos[3]
        if infos[4] != ' ':
            auteur.site = infos[4]
        auteur.bio = infos[5]    
        auteurs.append(auteur)
    
f.close()
print(len(auteurs))

f = open('../Data/ILE/oeuvres_ILE.csv',encoding="UTF-8")
document = f.read().split('\n')
for item in document:
    if item:
        infos = item.split('<>')
        oeuvre = Oeuvre(infos[0],infos[1])
        oeuvre.annee = infos[2]
        oeuvre.auteurs = infos[3].split(',')
        oeuvre.editeur = infos[4]
        oeuvre.lieuPublication = infos[5]
        oeuvre.isbn = infos[6]
        oeuvres.append(oeuvre)
        
print(len(oeuvres))
        
        
# for o in oeuvres.values():
#     f.write("{}<>{}<>{}<>".format(o.id, o.titre, o.annee))
#     f.write(",".join(o.auteurs) + "<>")
#     f.write("{}<>{}<>{}".format(o.editeur, o.lieuPublication, o.isbn))
#     f.write("\n")
# f.close()


    

http://recif.litterature.org/recherche/ecrivains/vonarburg-elisabeth-466/']
Ailleurs et au Japon['http://recif.litterature.org/recherche/ecrivains/vonarburg-elisabeth-466/']
Ailleurs ici['http://recif.litterature.org/recherche/ecrivains/vonarburg-elisabeth-466/']
Blood out of a stone['http://recif.litterature.org/recherche/ecrivains/vonarburg-elisabeth-466/']
Chanson pour une sirène['http://recif.litterature.org/recherche/ecrivains/vonarburg-elisabeth-466/']
Chroniques du pays des mères['http://recif.litterature.org/recherche/ecrivains/vonarburg-elisabeth-466/']
Chroniques du pays des mères['http://recif.litterature.org/recherche/ecrivains/vonarburg-elisabeth-466/']
Chroniques du pays des mères['http://recif.litterature.org/recherche/ecrivains/vonarburg-elisabeth-466/']
Comment écrire des histoires['http://recif.litterature.org/recherche/ecrivains/vonarburg-elisabeth-466/']
Les contes de la Chatte Rouge['http://recif.litterature.org/recherche/ecrivains/vonarburg-elisabeth-466/']
Contes

In [10]:
from rdflib import Graph, Namespace, Literal, URIRef
from rdflib import RDF, RDFS, OWL, XSD

BANQ = Namespace("http://banq.qc.ca/")
SCHEMA = Namespace("https://schema.org/")
DBO = Namespace("http://dbpedia.org/ontology/")
WDT = Namespace("http://www.wikidata.org/prop/direct/")
WD = Namespace("http://www.wikidata.org/entity/")
ILE_OEUVRE = Namespace("http://recif.litterature.org/ontologie/entite/oeuvre/")
ILE_AUTEUR = Namespace("http://recif.litterature.org/ontologie/entite/ecrivain/")
ILE_PROP = Namespace("http://recif.litterature.org/ontologie/propriete/")
ILE_CLASSE = Namespace("http://recif.litterature.org/ontologie/classe/")

def creer_URI_auteur(lien):
    lien_decompose = lien.split('/')
    if lien_decompose[-1] == "":
        return ILE_AUTEUR[lien_decompose[-2]]
    else:
        print('OUPS')
        return ILE_AUTEUR[lien_decompose[-1]]

def traiter_lieu_publication(lieu):
    return lieu

def traiter_bio(bio):
    return bio


grapheILE = Graph()

grapheILE.bind("schema","https://schema.org/", override=True)
grapheILE.bind("ile","http://recif.litterature.org/ontologie/propriete/", override=True)



for o in oeuvres:
    grapheILE.add((ILE_OEUVRE[o.id], RDF.type, ILE_CLASSE.oeuvre))   
    grapheILE.add((ILE_OEUVRE[o.id], SCHEMA.name, Literal(o.titre)))
    grapheILE.add((ILE_OEUVRE[o.id], ILE_PROP.lieuPublication, Literal(traiter_lieu_publication(o.lieuPublication))))
    grapheILE.add((ILE_OEUVRE[o.id], SCHEMA.bookEdition, Literal(o.editeur)))
    grapheILE.add((ILE_OEUVRE[o.id], SCHEMA.isbn, Literal(o.isbn)))    
    grapheILE.add((ILE_OEUVRE[o.id], SCHEMA.datePublished, Literal(o.annee)))        
    for a in o.auteurs:
        grapheILE.add((ILE_OEUVRE[o.id], SCHEMA.author, creer_URI_auteur(a)))

for a in auteurs:
    (nomFamille, prenom) = a.nom.split(", ")
    grapheILE.add((creer_URI_auteur(a.uri), RDF.type, ILE_CLASSE.ecrivain))
    grapheILE.add((creer_URI_auteur(a.uri), SCHEMA.name, Literal(a.nom)))
    grapheILE.add((creer_URI_auteur(a.uri), SCHEMA.givenName, Literal(prenom)))
    grapheILE.add((creer_URI_auteur(a.uri), SCHEMA.familyName, Literal(nomFamille)))   
    grapheILE.add((creer_URI_auteur(a.uri), ILE_PROP.uri, URIRef(a.uri)))

    if a.bio:
        grapheILE.add((creer_URI_auteur(a.uri), ILE_PROP.bio, Literal(traiter_bio(a.bio))))
        
    if a.site:
        grapheILE.add((creer_URI_auteur(a.uri), ILE_PROP.site, Literal(a.site)))

    if a.pseudonyme:
        grapheILE.add((creer_URI_auteur(a.uri), ILE_PROP.pseudonyme, Literal(a.pseudonyme)))
        
    for g in a.genres:
        grapheILE.add((creer_URI_auteur(a.uri), ILE_PROP.genre, Literal(g)))
    

NameError: name 'traiter_bio' is not defined

In [51]:
for s,p,o in grapheILE.triples((ILE_OEUVRE['4760'],None,None)):
    print(s,p,o)

print('----')    
for s,p,o in grapheILE.triples((ILE_AUTEUR['lavallee-loise-1135'],None,None)):
    print(s,p,o)

# for s,p,o in grapheILE.triples((None,ILE_PROP.site,None)):
#     print(s,p,o)
 
# for s,p,o in grapheILE.triples((None,ILE_PROP.lieuPublication,None)):
#     print(s,p,o)
 
    

http://recif.litterature.org/ontologie/entite/oeuvre/4760 https://schema.org/datePublished 1970
http://recif.litterature.org/ontologie/entite/oeuvre/4760 https://schema.org/isbn 
http://recif.litterature.org/ontologie/entite/oeuvre/4760 https://schema.org/name Elie, Elie, pourquoi
http://recif.litterature.org/ontologie/entite/oeuvre/4760 http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://recif.litterature.org/ontologie/classe/oeuvre
http://recif.litterature.org/ontologie/entite/oeuvre/4760 http://recif.litterature.org/ontologie/propriete/lieuPublication Montréal
http://recif.litterature.org/ontologie/entite/oeuvre/4760 https://schema.org/bookEdition Editions du Jour, Collection Les Poètes du jour ; M-24, 1970, 61 p. 20 cm.
----
http://recif.litterature.org/ontologie/entite/ecrivain/lavallee-loise-1135 http://recif.litterature.org/ontologie/propriete/genre Conte
http://recif.litterature.org/ontologie/entite/ecrivain/lavallee-loise-1135 http://recif.litterature.org/ontologie/propriet

In [625]:
fichier_ile = open("grapheILE.rdf","wb")
grapheILE.serialize(fichier_ile) 
fichier_ile.close()


In [50]:
print(len(grapheILE))

179160


À faire:
* Alignement avec graphe extrait à partir du dépôt légal
* Extraction des informations pertinentes à partir des biographies
* Nettoyage des informations exraites duy champ "édition"